In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle

from pympler import asizeof

from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#From signac
import signac
date_time_str = None
meth_name_str_list = [1, 2, 3, 4, 5]
study_id = "ep"
save_csv = True


criteria_dict = {"cs_name_val" : 9,
                 "param_name_str" : "t1t2t3t4",
                 "retrain_GP": 20,
                 "num_x_data": 5,
                 "num_val_pts": 0,
                 "lenscl": None}

df = pd.DataFrame()
job_list = []
for meth_name_val in meth_name_str_list:
    criteria_dict["meth_name_val"] = meth_name_val
    df_piece, jobs, name_cs_str, theta_true = get_study_data_signac(criteria_dict, study_id, save_csv)
    job_list += [job for job in jobs]
#     df_best = get_best_data(df_piece, study_id, name_cs_str, theta_true, param_name_str, date_time_str, save_csv)
    df = pd.concat([df, df_piece], ignore_index=True)
    
# df = df[df['EP Method Val'] == 4]

In [3]:
### Get Best Data from ep experiment
save_csv = True
df_best = get_best_data(df, study_id, name_cs_str, theta_true, job_list, date_time_str, save_csv)
df_best

# df_best["Min Obj Act"].iloc[2]
# save_path = "Results/" + study_id + "_study/" + cs_name + "/" + param_name_str + "/" +  study_id + "_study_best.csv"
# df_best.to_csv(save_path, header = True)
# #Get Mean Data from ep experiment
# df_mean = get_mean_data(df, study_id, name_cs_str, theta_true, param_name_str, date_time_str, save_csv)
# df_mean

# #Get Median Data from ep experiment
# df_median = get_median_data(df, study_id, name_cs_str, theta_true, param_name_str, date_time_str, save_csv)

# #Get Median of Best Data from ep experiment
# df_median_best = get_median_data(df_best, study_id, name_cs_str, theta_true, param_name_str, date_time_str, save_csv)
# df_median_best

,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,EP Method Val,BO Method,Max Evals,Total Run Time,L2 Norm Theta
3,1,1,1055.17969,1.0,0.0,"[10.53880839628684, 8.193143337657176, 208.532...",-386815.100059,26709.913361,"[13.964787571365758, 1e-05, 745.1753723454561,...",26709.913361,"[13.964787571365758, 1e-05, 745.1753723454561,...",0.513361,1,A1,2,1.026723,545.208814
11,0,1,6.961466,1.0,0.0,"[39.90045226576827, 2.583508934404272, 845.029...",7.29243,10.195943,"[1e-05, 1e-05, 1e-05, 1.0]",10.195943,"[1e-05, 1e-05, 1e-05, 1.0]",0.207689,1,B1,2,0.415377,200.999990
29,4,1,1055.17969,1.0,0.0,"[45.511334713401205, 1.5824168152454625, 130.7...",245.491488,521.23294,"[1e-05, 1e-05, 84.07505814662625, 0.1069799255...",521.23294,"[1e-05, 1e-05, 84.07505814662625, 0.1069799255...",80.599201,1,A2,2,161.198402,117.637746
39,4,1,6.961466,1.0,0.0,"[45.511334713401205, 1.5824168152454625, 130.7...",5.503262,6.256286,"[1e-05, 1e-05, 84.08944230913609, 0.1069543612...",6.256286,"[1e-05, 1e-05, 84.08944230913609, 0.1069543612...",79.412592,1,B2,2,158.825184,117.623571
49,4,1,1055.17969,1.0,0.0,"[45.511334713401205, 1.5824168152454625, 130.7...",245.491509,521.15819,"[1e-05, 1e-05, 84.05232789373923, 0.1070184076...",521.15819,"[1e-05, 1e-05, 84.05232789373923, 0.1070184076...",270.842002,1,C2,2,541.684003,117.660145


In [ ]:
#Get only the jobs which are the best
project = signac.get_project()
job_list_best = []
for meth_name_val in meth_name_str_list:
    #Get best ep data from previous results if possible    
    criteria_dict_ep = criteria_dict.copy()
    criteria_dict_ep["meth_name_val"] = meth_name_val
    criteria_dict_ep["sep_fact"] = 1.0
    meth_name = Method_name_enum(meth_name_val).name
    
    path_name = job_list[0].fn(study_id + "_study_best_all.csv")
    df_ep_best = pd.read_csv(path_name, header = 0)
    best_ep_enum_val = int(df_ep_best["EP Method Val"][(df_ep_best['BO Method'] == meth_name)])
    criteria_dict_ep["ep_enum_val"] = best_ep_enum_val
    
    #Get all jobs with that ep enum val
    jobs_best = project.find_jobs(criteria_dict_ep)
    job_list_best += [job for job in jobs_best]

for i in range(len(job_list_best)): 
    print(job_list_best[i].id)

In [ ]:
#Plot Muller potential
project = signac.get_project()
meth_name_str_list = [1,2, 3, 4, 5]
for meth_val in meth_name_str_list:
    criteria_dict["meth_name_val"] = meth_val
    jobs = project.find_jobs(criteria_dict) 
    for job in jobs:
        file_path = job.fn("BO_Results.gz")

    loaded_results = open_file_helper(file_path)
    meth_name = Method_name_enum(loaded_results[0].configuration["Method Name Enum Value"]).name


    run_num = df_best.loc[df_best['BO Method'].str.contains(meth_name), 'Run Number'].iloc[0] + 1
    bo_iter = df_best.loc[df_best['BO Method'].str.contains(meth_name), 'BO Iter'].iloc[0] 
    trained_hps = loaded_results[run_num-1].list_gp_emulator_class[bo_iter-1].trained_hyperparams
    print("Method: ", meth_name)
    print("Run Number: ", run_num)
    print("BO Iter Number: ", bo_iter)
    print("Trained Hyperparameters: ", trained_hps)
    x_val_num = 20
    theta_choice = None
    theta_true = loaded_results[run_num-1].simulator_class.theta_true
    theta_choice = theta_true  
#     theta_choice = np.array([-0.12334013, -1.97193209, -9.58736122,  0.70718457]) #(CS2_4 a 2A Set HPs)
#     theta_choice = np.array([-1.25179735, -1.30799798, -6.38989644,  0.69741102]) #(CS2_4 a 2A Trained HPs)   
#     theta_choice = np.array([-1.52838065,  0.76858661, -8.32826196, -0.15766997]) #(CS2_4 a 2C Set HPs)
#     theta_choice = np.array([-0.002612917, 0.493312186, 1.491001765, 0.999614708])  #(CS2_4 y0 2A Trained HPs) 
#     theta_choice = np.array([ 0.9346134,  -0.14312267,  0.65663397, -1.46321351])  #(CS2_4 y0 2C Trained HPs) 

    seed = loaded_results[0].configuration["Seed"]

    title_fontsize = 24
    other_fontsize = 20
    xbins = 4
    ybins = 5
    zbins = 900
    save_path = None
    cmap = "autumn"
    save_path = None

    data_vals = compare_muller_heat_map(file_path, run_num, bo_iter, x_val_num, theta_choice, seed)
    test_mesh, y_sim, gp_mean, gp_var, theta_value, train_X, idcs_to_plot = data_vals
    title = "Heat Map For Theta = " + str(np.round(theta_value,1))

#     z = [y_sim]
#     z_titles = ["y_sim"]
    
    z = [y_sim,  gp_mean, gp_var]
#     print(np.min(y_sim), np.max(y_sim))
    z_titles = ["y_sim" , "y_gp", "y_var"]
    
    cmap = "jet"
    integer = 200
    levels = [integer,integer,integer]
    plot_axis_names = ["X1", "X2"]
    try:
        plot_heat_maps(test_mesh, None, None, None, train_X, plot_axis_names, levels, idcs_to_plot, z, 
                   z_titles, xbins, ybins, zbins, title, title_fontsize, other_fontsize, cmap, save_path)
    except:
        pass
# print("Max EI Theta", theta_choice)